In [2]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from pybaseball import pitching_stats
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [3]:
#Read in data
data20_22 = pd.read_csv("Statcast_2020_to_2023.csv")
data_today = statcast()

#Select columns
data20_22 = data20_22[['player_name','pitch_type','release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','vz0','ax','ay','az','game_year','delta_run_exp']]
data_today = data_today[['player_name','pitch_type','release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','vz0','ax','ay','az','game_year','delta_run_exp']]

#Drop na values
data20_22 = data20_22.dropna()
data_today = data_today.dropna()

#Flip to get view from pitcher
data20_22['pfx_x'] = data20_22['pfx_x'] * -1
data_today['pfx_x'] = data_today['pfx_x'] * -1

#Change to inches
data20_22['pfx_x'] = data20_22['pfx_x'] * 12
data20_22['pfx_z'] = data20_22['pfx_z'] * 12
data_today['pfx_x'] = data_today['pfx_x'] * 12
data_today['pfx_z'] = data_today['pfx_z'] * 12

#Remove unnecessary pitches
data20_22 = data20_22[data20_22.pitch_type != 'PO']
data_today = data_today[data_today.pitch_type != 'PO']

#Retag pitches
data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FF','FA'], 'Fastball')
data_today['pitch_type'] = data_today['pitch_type'].replace(['FF','FA'], 'Fastball')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['SI'], 'Sinker')
data_today['pitch_type'] = data_today['pitch_type'].replace(['SI'], 'Sinker')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FC'], 'Cutter')
data_today['pitch_type'] = data_today['pitch_type'].replace(['FC'], 'Cutter')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['SL'], 'Slider')
data_today['pitch_type'] = data_today['pitch_type'].replace(['SL'], 'Slider')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['ST'], 'Sweeper')
data_today['pitch_type'] = data_today['pitch_type'].replace(['ST'], 'Sweeper')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['CU','CS','SV','KC'], 'Curveball')
data_today['pitch_type'] = data_today['pitch_type'].replace(['CU','CS','SV','KC'], 'Curveball')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['CH','FO','EP','KN','SC'], 'Changeup')
data_today['pitch_type'] = data_today['pitch_type'].replace(['CH','FO','EP','KN','SC'], 'Changeup')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FS'], 'Splitter')
data_today['pitch_type'] = data_today['pitch_type'].replace(['FS'], 'Splitter')


start_dt 2024-06-13
end_dt 2024-06-14
This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:02<00:00,  1.33s/it]
/Users/maxwassarman/miniconda3/envs/baseball/lib/python3.12/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


In [4]:
#caculate vaa and haa
def approach_angles(df):
    yf = 17/12
    y0 = 50
    df['vy_f'] = -np.sqrt(df['vy0']**2 - (2 * df['ay'] * y0 - yf))
    df['t'] = (df['vy_f'] - df['vy0']) / df['ay']
    df['vz_f'] = df['vz0'] + (df['az'] * df['t'])
    df['vx_f'] = df['vx0'] + (df['ax'] * df['t'])

    df['vaa'] = -np.arctan(df['vz_f'] / df['vy_f']) * (180 / np.pi)
    df['haa'] = -np.arctan(df['vx_f'] / df['vy_f']) * (180 / np.pi)
    return df

data20_22 = approach_angles(data20_22)
data_today = approach_angles(data_today)


#caculate estimated axis differential and estimated spin efficiency
def axis_differential(df):
    df['calc_spin'] = np.arctan2(df['pfx_z'],df['pfx_x']) * (180 / np.pi)
    df['axis_dif'] = np.abs(df['calc_spin'] - df['spin_axis'])
    return df

data20_22 = axis_differential(data20_22)
data_today = axis_differential(data_today)


def find_most_common_fastball(df):
    fastball_types = df[df['pitch_type'].isin(['Fastball', 'Sinker', 'Cutter'])]
    most_common_fastball = fastball_types.groupby(['player_name', 'pitch_type']).size().reset_index(name='count')
    most_common_fastball = most_common_fastball.loc[most_common_fastball.groupby('player_name')['count'].idxmax()]
    most_common_fastball = most_common_fastball[['player_name', 'pitch_type']]
    most_common_fastball.rename(columns={'pitch_type': 'most_common_fastball'}, inplace=True)
    return most_common_fastball

def add_velocity_and_movement_diff(df):
    most_common_fastball = find_most_common_fastball(df)
    
    df = df.merge(most_common_fastball, on='player_name', how='left')
    
    avg_stats = df[df['pitch_type'] == df['most_common_fastball']].groupby('player_name')[['release_speed', 'pfx_x', 'pfx_z']].mean().reset_index()
    avg_stats.rename(columns={
        'release_speed': 'avg_fastball_velocity',
        'pfx_x': 'avg_fastball_pfx_x',
        'pfx_z': 'avg_fastball_pfx_z'
    }, inplace=True)
    
    df = df.merge(avg_stats, on='player_name', how='left')
    
    df['velocity_diff'] = df['avg_fastball_velocity'] - df['release_speed']
    df['horizontal_movement_diff'] = df['avg_fastball_pfx_x'] - df['pfx_x']
    df['vertical_movement_diff'] = df['avg_fastball_pfx_z'] - df['pfx_z']
    
    return df

data20_22 = add_velocity_and_movement_diff(data20_22)
data_today = add_velocity_and_movement_diff(data_today)

In [5]:
#Seperate pitches for different models
fastball_20_22 = data20_22[data20_22.pitch_type.isin(['Fastball'])]
fastball_23 = data_today[data_today.pitch_type.isin(['Fastball'])]

sinker_20_22 = data20_22[data20_22.pitch_type.isin(['Sinker'])]
sinker_23 = data_today[data_today.pitch_type.isin(['Sinker'])]

cutter_20_22 = data20_22[data20_22.pitch_type.isin(['Cutter'])]
cutter_23 = data_today[data_today.pitch_type.isin(['Cutter'])]

slider_20_22 = data20_22[data20_22.pitch_type.isin(['Slider'])]
slider_23 = data_today[data_today.pitch_type.isin(['Slider'])]

sweeper_20_22 = data20_22[data20_22.pitch_type.isin(['Sweeper'])]
sweeper_23 = data_today[data_today.pitch_type.isin(['Sweeper'])]

curveball_20_22 = data20_22[data20_22.pitch_type.isin(['Curveball'])]
curveball_23 = data_today[data_today.pitch_type.isin(['Curveball'])]

changeup_20_22 = data20_22[data20_22.pitch_type.isin(['Changeup'])]
changeup_23 = data_today[data_today.pitch_type.isin(['Changeup'])]

splitter_20_22 = data20_22[data20_22.pitch_type.isin(['Splitter'])]
splitter_23 = data_today[data_today.pitch_type.isin(['Splitter'])]


In [7]:
features = ['release_speed','release_pos_x','release_pos_z','release_extension','pfx_x', 'pfx_z','game_year', 'axis_dif', 'velocity_diff', 'horizontal_movement_diff', 'vertical_movement_diff']
#Split into train and test

fastball_x = fastball_20_22[features]
fastball_y = fastball_20_22['delta_run_exp']

x_train_fastball, x_test_fastball, y_train_fastball, y_test_fastball = train_test_split(fastball_x, fastball_y, train_size = 0.75, random_state = 12345)

sinker_x = sinker_20_22[features]
sinker_y = sinker_20_22['delta_run_exp']

x_train_sinker, x_test_sinker, y_train_sinker, y_test_sinker = train_test_split(sinker_x, sinker_y, train_size=0.75, random_state=12345)

cutter_x = cutter_20_22[features]
cutter_y = cutter_20_22['delta_run_exp']

x_train_cutter, x_test_cutter, y_train_cutter, y_test_cutter = train_test_split(cutter_x, cutter_y, train_size=0.75, random_state=12345)

slider_x = slider_20_22[features]
slider_y = slider_20_22['delta_run_exp']

x_train_slider, x_test_slider, y_train_slider, y_test_slider = train_test_split(slider_x, slider_y, train_size=0.75, random_state=12345)

sweeper_x = sweeper_20_22[features]
sweeper_y = sweeper_20_22['delta_run_exp']

x_train_sweeper, x_test_sweeper, y_train_sweeper, y_test_sweeper = train_test_split(sweeper_x, sweeper_y, train_size=0.75, random_state=12345)

curveball_x = curveball_20_22[features]
curveball_y = curveball_20_22['delta_run_exp']

x_train_curveball, x_test_curveball, y_train_curveball, y_test_curveball = train_test_split(curveball_x, curveball_y, train_size=0.75, random_state=12345)

changeup_x = changeup_20_22[features]
changeup_y = changeup_20_22['delta_run_exp']

x_train_changeup, x_test_changeup, y_train_changeup, y_test_changeup = train_test_split(changeup_x, changeup_y, train_size=0.75, random_state=12345)

splitter_x = splitter_20_22[features]
splitter_y = splitter_20_22['delta_run_exp']

x_train_splitter, x_test_splitter, y_train_splitter, y_test_splitter = train_test_split(splitter_x, splitter_y, train_size=0.75, random_state=12345)

In [8]:
#Set params for catboost models (Random ones right now)
params = {
    'iterations': 500,
    'depth': 5,
    'learning_rate': 0.11, #manual checks with FIP corr both predictive and descriptive
    'random_seed': 12345    
}

#Train each model

#Fastball
fastball_model = CatBoostRegressor(**params)
fastball_model.fit(x_train_fastball, y_train_fastball)

#Sinker
sinker_model = CatBoostRegressor(**params)
sinker_model.fit(x_train_sinker, y_train_sinker)

#Cutter
cutter_model = CatBoostRegressor(**params)
cutter_model.fit(x_train_cutter, y_train_cutter)

#Slider
slider_model = CatBoostRegressor(**params)
slider_model.fit(x_train_slider, y_train_slider)

#Sweeper
sweeper_model = CatBoostRegressor(**params)
sweeper_model.fit(x_train_sweeper, y_train_sweeper)

#Curveball
curveball_model = CatBoostRegressor(**params)
curveball_model.fit(x_train_curveball, y_train_curveball)

#Changeup
changeup_model = CatBoostRegressor(**params)
changeup_model.fit(x_train_changeup, y_train_changeup)

#Splitter
splitter_model = CatBoostRegressor(**params)
splitter_model.fit(x_train_splitter, y_train_splitter)



0:	learn: 0.2412655	total: 78.2ms	remaining: 39s
1:	learn: 0.2412441	total: 98.4ms	remaining: 24.5s
2:	learn: 0.2412333	total: 140ms	remaining: 23.1s
3:	learn: 0.2412196	total: 174ms	remaining: 21.6s
4:	learn: 0.2412051	total: 195ms	remaining: 19.3s
5:	learn: 0.2411924	total: 210ms	remaining: 17.3s
6:	learn: 0.2411851	total: 228ms	remaining: 16s
7:	learn: 0.2411763	total: 246ms	remaining: 15.1s
8:	learn: 0.2411690	total: 261ms	remaining: 14.2s
9:	learn: 0.2411610	total: 276ms	remaining: 13.5s
10:	learn: 0.2411541	total: 290ms	remaining: 12.9s
11:	learn: 0.2411482	total: 307ms	remaining: 12.5s
12:	learn: 0.2411425	total: 324ms	remaining: 12.1s
13:	learn: 0.2411363	total: 340ms	remaining: 11.8s
14:	learn: 0.2411325	total: 357ms	remaining: 11.6s
15:	learn: 0.2411277	total: 375ms	remaining: 11.4s
16:	learn: 0.2411223	total: 393ms	remaining: 11.2s
17:	learn: 0.2411171	total: 410ms	remaining: 11s
18:	learn: 0.2411108	total: 431ms	remaining: 10.9s
19:	learn: 0.2411060	total: 449ms	remaining: 

In [9]:
#Predictions
fastball_23['pred'] = fastball_model.predict(fastball_23[features])

sinker_23['pred'] = sinker_model.predict(sinker_23[features])

cutter_23['pred'] = cutter_model.predict(cutter_23[features])

slider_23['pred'] = slider_model.predict(slider_23[features])

sweeper_23['pred'] = sweeper_model.predict(sweeper_23[features])

curveball_23['pred'] = curveball_model.predict(curveball_23[features])

changeup_23['pred'] = changeup_model.predict(changeup_23[features])

splitter_23['pred'] = splitter_model.predict(splitter_23[features])

/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/2925554147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_23['pred'] = fastball_model.predict(fastball_23[features])
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/2925554147.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinker_23['pred'] = sinker_model.predict(sinker_23[features])
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/2925554147.py:6: SettingWithCopyWarning: 
A value is trying t

In [10]:
def scale_and_score(df):
    df['scaled'] = df['pred'] - df['pred'].max()
    df['scaled'] = abs(df['scaled'])
    df['scaled'] = df['scaled'] - df['scaled'].mean()
    df['scaled'] = df['scaled'] / df['scaled'].std()
    df['score'] = df['scaled'] * 10 + 100
    return df

scale_and_score(fastball_23)
scale_and_score(sinker_23)
scale_and_score(cutter_23)
scale_and_score(slider_23)
scale_and_score(sweeper_23)
scale_and_score(curveball_23)
scale_and_score(changeup_23)
scale_and_score(splitter_23)

def calculate_mean_scores(df):
    mean_scores = df.groupby('player_name')['score'].mean().reset_index()
    mean_scores.rename(columns={'score': 'mean_scores'}, inplace=True)
    mean_scores = mean_scores.sort_values(by='mean_scores', ascending=False)
    mean_scores['mean_scores'] = mean_scores['mean_scores'].round(2)
    return mean_scores

mean_scores_fastball = calculate_mean_scores(fastball_23)
mean_scores_sinker = calculate_mean_scores(sinker_23)
mean_scores_cutter = calculate_mean_scores(cutter_23)
mean_scores_slider = calculate_mean_scores(slider_23)
mean_scores_sweeper = calculate_mean_scores(sweeper_23)
mean_scores_curveball = calculate_mean_scores(curveball_23)
mean_scores_changeup = calculate_mean_scores(changeup_23)
mean_scores_splitter = calculate_mean_scores(splitter_23)

combined_scores = mean_scores_fastball
combined_scores = combined_scores.merge(mean_scores_sinker, how = 'left', on = 'player_name', suffixes=('','_sinker'))
combined_scores = combined_scores.merge(mean_scores_cutter, how = 'left', on = 'player_name', suffixes=('','_cutter'))
combined_scores = combined_scores.merge(mean_scores_slider, how = 'left', on = 'player_name', suffixes=('','_slider'))
combined_scores = combined_scores.merge(mean_scores_sweeper, how = 'left', on = 'player_name', suffixes=('','_sweeper'))
combined_scores = combined_scores.merge(mean_scores_curveball, how = 'left', on = 'player_name', suffixes=('','_curveball'))
combined_scores = combined_scores.merge(mean_scores_changeup, how = 'left', on = 'player_name', suffixes=('','_changeup'))
combined_scores = combined_scores.merge(mean_scores_splitter, how = 'left', on = 'player_name', suffixes=('','_splitter'))



combined_scores = combined_scores.rename(columns = {
    'mean_scores': 'fastball',
    'mean_scores_sinker': 'sinker',
    'mean_scores_cutter': 'cutter',
    'mean_scores_slider': 'slider',
    'mean_scores_sweeper': 'sweeper',
    'mean_scores_curveball': 'curveball',
    'mean_scores_changeup': 'changeup',
    'mean_scores_splitter': 'splitter'
    }
)

pitch_counts = data_today.groupby(['player_name', 'pitch_type']).size().unstack(fill_value=0).reset_index()

combined_data = combined_scores.merge(pitch_counts, how='left', on='player_name')
combined_data.fillna(0, inplace=True)

combined_data['weighted_fastball'] = combined_data['fastball'] * combined_data['Fastball']
combined_data['weighted_sinker'] = combined_data['sinker'] * combined_data['Sinker']
combined_data['weighted_cutter'] = combined_data['cutter'] * combined_data['Cutter']
combined_data['weighted_slider'] = combined_data['slider'] * combined_data['Slider']
combined_data['weighted_sweeper'] = combined_data['sweeper'] * combined_data['Sweeper']
combined_data['weighted_curveball'] = combined_data['curveball'] * combined_data['Curveball']
combined_data['weighted_changeup'] = combined_data['changeup'] * combined_data['Changeup']
combined_data['weighted_splitter'] = combined_data['splitter'] * combined_data['Splitter']

combined_data['total_weighted_score'] = (
    combined_data['weighted_fastball'] +
    combined_data['weighted_sinker'] +
    combined_data['weighted_cutter'] +
    combined_data['weighted_slider'] +
    combined_data['weighted_sweeper'] +
    combined_data['weighted_curveball'] +
    combined_data['weighted_changeup'] +
    combined_data['weighted_splitter']
)

combined_data['total_pitches'] = (
    combined_data['Fastball'] +
    combined_data['Sinker'] +
    combined_data['Cutter'] +
    combined_data['Slider'] +
    combined_data['Sweeper'] +
    combined_data['Curveball'] +
    combined_data['Changeup'] +
    combined_data['Splitter']
)

combined_data['overall'] = combined_data['total_weighted_score'] / combined_data['total_pitches']

combined_data['overall'] = combined_data['overall'].round(2)

combined_data = combined_data[['player_name', 'overall', 'fastball', 'sinker', 'cutter', 'slider', 'sweeper', 'curveball', 'changeup', 'splitter']]

combined_data = combined_data.sort_values(by='overall', ascending=False)

combined_data.to_csv("day_pitch_scores.csv", index=False)
combined_data

/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/705413268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scaled'] = df['pred'] - df['pred'].max()
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/705413268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scaled'] = abs(df['scaled'])
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_47979/705413268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,player_name,overall,fastball,sinker,cutter,slider,sweeper,curveball,changeup,splitter
42,"Jax, Griffin",114.62,99.46,116.70,0.00,0.00,105.38,0.00,131.04,0.0
1,"Fairbanks, Pete",112.30,109.53,0.00,0.00,117.84,0.00,0.00,0.00,0.0
33,"Díaz, Edwin",110.98,101.01,0.00,0.00,119.70,0.00,0.00,0.00,0.0
0,"Vesia, Alex",109.14,112.26,0.00,0.00,101.34,0.00,0.00,0.00,0.0
19,"Alcala, Jorge",108.30,102.92,124.71,0.00,110.94,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
60,"Ginkel, Kevin",95.51,95.56,95.36,0.00,0.00,0.00,0.00,0.00,0.0
18,"Fernandez, Ryan",94.20,102.95,89.49,0.00,96.62,0.00,0.00,0.00,0.0
30,"Schreiber, John",92.57,101.30,116.14,50.15,0.00,60.65,0.00,97.17,0.0
67,"Muñoz, Roddery",92.00,87.52,91.04,98.62,85.03,0.00,0.00,78.96,0.0


In [11]:
def get_top_individual_pitches(pitch_df, pitch_type):
    top_pitches = pitch_df[['player_name', 'pitch_type', 'release_speed','release_pos_x','release_pos_z','release_extension','pfx_x', 'pfx_z','score']]
    top_pitches = top_pitches.sort_values(by='score', ascending=False)
    top_pitches['pitch_type'] = pitch_type
    return top_pitches

top_fastball_pitches = get_top_individual_pitches(fastball_23, 'Fastball')
top_sinker_pitches = get_top_individual_pitches(sinker_23, 'Sinker')
top_cutter_pitches = get_top_individual_pitches(cutter_23, 'Cutter')
top_slider_pitches = get_top_individual_pitches(slider_23, 'Slider')
top_sweeper_pitches = get_top_individual_pitches(sweeper_23, 'Sweeper')
top_curveball_pitches = get_top_individual_pitches(curveball_23, 'Curveball')
top_changeup_pitches = get_top_individual_pitches(changeup_23, 'Changeup')
top_splitter_pitches = get_top_individual_pitches(splitter_23, 'Splitter')

top_individual_pitches = pd.concat([
    top_fastball_pitches,
    top_sinker_pitches,
    top_cutter_pitches,
    top_slider_pitches,
    top_sweeper_pitches,
    top_curveball_pitches,
    top_changeup_pitches,
    top_splitter_pitches
])
top_individual_pitches = top_individual_pitches.sort_values(by='score', ascending=False)

top_individual_pitches.to_csv("top_individual_pitches.csv", index=False)
top_individual_pitches

,player_name,pitch_type,release_speed,release_pos_x,release_pos_z,release_extension,pfx_x,pfx_z,score
2882,"Miller, Tyson",Sweeper,79.4,-3.76,4.55,7.4,-11.52,12.84,151.355746
2881,"Miller, Tyson",Sweeper,79.2,-3.93,4.55,7.0,-12.0,12.0,149.255295
1192,"Holmes, Clay",Sinker,97.3,-0.82,6.37,6.1,19.44,1.44,140.318599
1757,"Jax, Griffin",Changeup,93.7,-0.63,5.56,6.4,16.92,1.08,139.602990
1680,"Yarbrough, Ryan",Curveball,71.7,3.45,4.74,6.2,16.08,9.84,137.635692
...,...,...,...,...,...,...,...,...,...
1253,"Schreiber, John",Cutter,89.3,-2.64,4.07,6.5,7.68,5.52,16.574287
1640,"Yarbrough, Ryan",Curveball,72.7,3.46,4.56,6.5,17.76,7.44,7.725568
1659,"Yarbrough, Ryan",Fastball,87.1,3.26,4.88,6.2,-14.4,6.84,3.789172
1653,"Yarbrough, Ryan",Fastball,87.7,3.15,4.92,6.4,-15.6,7.56,-6.310349
